In [25]:
import sqlite3
import pandas as pd

In [26]:
conn = sqlite3.connect('hackathon.db')
pd.read_sql('''
            
            -- set up staging table for dim_senders
WITH stg_senders AS(
  SELECT message_id, send_date, send_time, content, sms_messages.sender_id, name AS sender_name, username AS sender_username,
  phone AS sender_phone, product_name AS sender_product_name, serial_number AS sender_serial_number,
  model_number AS sender_model_number, software_version AS sender_software_version
  FROM senders
  JOIN sms_messages ON senders.sender_id = sms_messages.sender_id
  JOIN devices ON senders.sender_id = devices.device_id),
  
  -- set up staging table for dim_receivers
  stg_receivers AS(
      SELECT message_id AS message_id2, send_date AS send_date2, send_time AS send_time2, content AS content2, sms_messages.receiver_id, 
      name AS receiver_name, username AS receiver_username, phone AS receiver_phone, product_name AS receiver_product_name, 
      serial_number AS receiver_serial_number, model_number AS receiver_model_number, software_version AS receiver_software_version
      FROM receivers
      JOIN sms_messages ON receivers.receiver_id = sms_messages.receiver_id
      JOIN devices ON receivers.receiver_id = devices.device_id),
 
 -- set up staging table for fact_sms_messages
 stg_sms_messages1 AS(
  SELECT *
  FROM stg_senders
  JOIN stg_receivers ON stg_receivers.message_id2 = stg_senders.message_id),

-- define dim_senders
dim_senders AS(
  SELECT DISTINCT sender_id, sender_name, sender_username, sender_phone, sender_product_name,
  sender_serial_number, sender_model_number, sender_software_version
  FROM stg_sms_messages1),
  
-- view difference after dim_senders
 stg_sms_messages2 AS(
  SELECT message_id, send_date, send_time, content, sender_id, send_date2, send_time2, content2, receiver_id, receiver_name, receiver_username, 
  receiver_phone, receiver_product_name, receiver_serial_number, receiver_model_number, receiver_software_version
  FROM stg_sms_messages1),
  
  -- define dim_receivers
  dim_receivers AS(
    SELECT DISTINCT receiver_id, receiver_name, receiver_username, receiver_phone, receiver_product_name,
    receiver_serial_number, receiver_model_number, receiver_software_version
    FROM stg_sms_messages2),
 
 -- view difference after dim_receivers
 stg_sms_messages3 AS(
   SELECT message_id, send_date, send_time, content, sender_id, send_date2, send_time2, content2, receiver_id
   FROM stg_sms_messages2),
 
 -- define fact_sms_messages
 fact_sms_messages AS(
   SELECT message_id, send_date, send_time, content, sender_id, receiver_id
   FROM stg_sms_messages3)
   
   SELECT *
   FROM fact_sms_messages AS fact
   JOIN dim_senders ON fact.sender_id = dim_senders.sender_id
   JOIN dim_receivers ON fact.receiver_id = dim_receivers.receiver_id
   LIMIT 5''', conn)

,message_id,send_date,send_time,content,sender_id,receiver_id,sender_name,sender_username,sender_phone,sender_product_name,sender_serial_number,sender_model_number,sender_software_version,receiver_name,receiver_username,receiver_phone,receiver_product_name,receiver_serial_number,receiver_model_number,receiver_software_version
0,1,2023-03-09,10:38:08,:),628,586,Adella Spires,aspireshf,9891192006,Apple iPhone 12 Plus,3WBZ53RY7V,96C2D8E,15.9.0,Peadar Gumly,pgumlyg9,8273506116,Samsung Galaxy S21,63LQVVNKW9,87B27Y/,15.9.0
1,2,2023-02-14,16:09:49,Can you pick up milk on your way home?,878,122,Gracie Barkworth,gbarkworthod,3482233269,Apple iPhone 13 Plus,895KHREAHE,WCUNGVH,15.9.0,Micky Manntschke,,4222908146,Samsung Galaxy S23,CHB55WSWDZ,SWHHP82,16.1.0
2,3,2023-01-28,08:29:48,Can you pick up milk on your way home?,135,310,Peterus Boyer,pboyer3q,3657041760,Samsung Galaxy S22,REMV4R3WV9,GEPB5Y3,12.1.3,Kirstyn Butchard,kbutchard8l,3974501189,Apple iPhone SE,D8DCU2SANR,FXRUJUN,16.0.2
3,4,2023-01-16,20:14:04,Can we talk?,743,616,Ferdinanda Cattell,,4934071352,Samsung Galaxy S10,RMB5D2XT5Y,TA4WGBU,16.1.0,Elfie Laurenz,elaurenzh3,4236885864,Samsung Galaxy S22,5EPKN5XDW3,26VXLGV,12.1.3
4,5,2023-03-28,22:14:06,LOL,330,321,Mona Gethouse,,5214346614,Apple iPhone 12 Plus,GUYUYCUX7S,6/K5JTT,15.9.4,Lora Sayburn,,9927949024,Apple iPhone 12 Plus,MRWQYH3LAE,RMKAS9T,16.0.2
